In [102]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('p14v2irene.csv')
df1 = df1.fillna(0)
df2 = pd.read_csv('p14v2.csv')
df2 = df2.fillna(0)
df1.rename(columns=lambda x: x.strip(), inplace=True)
df2.rename(columns=lambda x: x.strip(), inplace=True)

columns_to_delete = [0, 2, 3]
df1 = df1.drop(df1.columns[columns_to_delete], axis=1)
df2 = df2.drop(df2.columns[columns_to_delete], axis=1)

values_in_df1 = set(df1['Timestamp'])
df2_filtered = df2[df2['Timestamp'].isin(values_in_df1)]
df2 = df2_filtered

df1 = df1.drop(df1.index[20:1000])
df1 = df1.drop(df1.columns[[37]], axis=1)
print(df1)

df2 = df2.drop(df2.index[20:1000])
#df2 = df2.drop(df2.columns[[41]], axis=1)
#print(df2)


   Timestamp Speaker                                          Utterance  \
0   41:42:00   Robot  can you count the number of petes friends in t...   
1   41:46:00   Child                                      1, 2, 3, 4, 5   
2   41:57:00  Parent  And then I can hit this if I wanted to answer....   
3   42:04:00   Robot  Four of petes friends are in animal shaped flo...   
4   43:18:00   Robot  the alligator has some sharp teeth. Can you co...   
5   43:23:00   Child                          1, 2, 3, 4, 5, 6, 7, 8, 9   
6   43:32:00  Parent  think you counted one twice. Okay, let's see. ...   
7   46:09:00   Robot                 Are there more cats or alligators?   
8   46:13:00   Child                                          more cat.   
9   46:14:00  Parent                  And see how many cats do you see?   
10  46:16:00   Child                                               1, 2   
11  46:17:00  Parent                And how many alligators Do you see?   
12  46:19:00   Child     

In [105]:
#COMPARE
df_compare = df1.compare(df2, align_axis=1, keep_shape=True, keep_equal=True)

# style func to highlight the cells that are different
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('self', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

df_compare2 = df_compare[(df1 != df2).any(1)].style.apply(highlight_diff, axis=None)
writer = pd.ExcelWriter('v2_P14_comparison_jw.xlsx', engine='xlsxwriter')
df_compare2.to_excel(writer, sheet_name='Differences')
writer.close()  


ValueError: Can only compare identically-labeled DataFrame objects

In [104]:
from sklearn.metrics import cohen_kappa_score

# COHEN KAPPA VALUES
arrayColumns = df1.columns
arrayr1all = []
arrayr2all = []

for x in range(3, 37):
    arrayr1all.append(df1[arrayColumns[x]].tolist())
    arrayr2all.append(df2[arrayColumns[x]].tolist())

data = []
arrayColumns = arrayColumns[3:]

for i in range(len(arrayr1all)):
    data.append([arrayColumns[i], cohen_kappa_score(arrayr1all[i], arrayr2all[i])])

arrayr1all_f = np.array(arrayr1all).flatten()
arrayr2all_f = np.array(arrayr2all).flatten()
data.append(['Overall Cohen-Kappa Value', cohen_kappa_score(arrayr1all_f, arrayr2all_f)])

summary = pd.DataFrame(data)
summary.to_csv('p14v2overall_kappa_jw', sep='\t', index=False)


KeyError: '[parent] Read material content'